In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

# Load the extracted audio features from CSV

data = pd.read_csv('DATASET-LA.csv')

# Assuming your CSV has columns like 'feature_1', 'feature_2', ..., 'label'
X = data.iloc[:, :-1].values
y = data.iloc[:, -1].values

# Encode the labels
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Reshape the input data for TDNN (assuming time dimension is the second axis)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Define the TDNN model
model = Sequential([
    layers.Conv1D(64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1)),
    layers.MaxPooling1D(pool_size=2),
    layers.Conv1D(128, kernel_size=3, activation='relu'),
    layers.MaxPooling1D(pool_size=2),
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

# Evaluate the model on the test set
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

# Convert back to original labels
y_pred = label_encoder.inverse_transform(y_pred.flatten())
y_test = label_encoder.inverse_transform(y_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

Epoch 1/10
2270/2270 [==============================] - 25s 10ms/step - loss: 0.5749 - accuracy: 0.8957 - val_loss: 0.3339 - val_accuracy: 0.8980
Epoch 2/10
2270/2270 [==============================] - 22s 10ms/step - loss: 0.3395 - accuracy: 0.8984 - val_loss: 0.3311 - val_accuracy: 0.8981
Epoch 3/10
2270/2270 [==============================] - 25s 11ms/step - loss: 0.3323 - accuracy: 0.8984 - val_loss: 0.3294 - val_accuracy: 0.8981
Epoch 4/10
2270/2270 [==============================] - 21s 9ms/step - loss: 0.3294 - accuracy: 0.8983 - val_loss: 0.3295 - val_accuracy: 0.8981
Epoch 5/10
2270/2270 [==============================] - 23s 10ms/step - loss: 0.3294 - accuracy: 0.8984 - val_loss: 0.3293 - val_accuracy: 0.8981
Epoch 6/10
2270/2270 [==============================] - 22s 9ms/step - loss: 0.3286 - accuracy: 0.8984 - val_loss: 0.3292 - val_accuracy: 0.8981
Epoch 7/10
2270/2270 [==============================] - 22s 10ms/step - loss: 0.3285 - accuracy: 0.8984 - val_loss: 0.3292 - v

In [ ]:
from sklearn.metrics import confusion_matrix

# Assuming y_test and y_pred are your true labels and predicted labels
conf_matrix = confusion_matrix(y_test, y_pred)

print("Confusion Matrix:")
print(conf_matrix)


Confusion Matrix:
[[    0  3700]
 [    0 32614]]


In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

precision = precision_score(y_test, y_pred, average='binary', pos_label='spoof')
recall = recall_score(y_test, y_pred, average='binary', pos_label='spoof')
f1 = f1_score(y_test, y_pred, average='binary', pos_label='spoof')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')

Precision: 0.8981109214077215
Recall: 1.0
F1 Score: 0.9463207985143918
